In [1]:
%load_ext sql

In [2]:
%sql sqlite:///chinook.db

__List each employee's first and last name along with their boss's first and last name__

In [55]:
%%sql
SELECT e.firstname || ' ' ||  e.lastname AS workers
, m.firstname || ' ' || m.lastname AS boss
FROM employees AS e 
LEFT JOIN employees AS m 
    ON m.employeeid = e.reportsto 

 * sqlite:///chinook.db
Done.


[('Andrew Adams', None),
 ('Nancy Edwards', 'Andrew Adams'),
 ('Jane Peacock', 'Nancy Edwards'),
 ('Margaret Park', 'Nancy Edwards'),
 ('Steve Johnson', 'Nancy Edwards'),
 ('Michael Mitchell', 'Andrew Adams'),
 ('Robert King', 'Michael Mitchell'),
 ('Laura Callahan', 'Michael Mitchell')]

__Name employee and boss who are in different city__

In [92]:
%%sql
SELECT e.firstname AS employee 
, m.firstname AS boss 
, e.city AS emp_city 
, m.city AS boss_city
FROM employees AS e 
JOIN employees AS m 
    ON e.reportsto = m.employeeid 
WHERE e.city <> m.city

 * sqlite:///chinook.db
Done.


[('Nancy', 'Andrew', 'Calgary', 'Edmonton'),
 ('Michael', 'Andrew', 'Calgary', 'Edmonton'),
 ('Robert', 'Michael', 'Lethbridge', 'Calgary'),
 ('Laura', 'Michael', 'Lethbridge', 'Calgary')]

__Find managers who have at least 2 employees directly reporting to them__

In [42]:
%%sql 
SELECT m.firstname, m.lastname 
FROM employees AS m  
JOIN employees AS e
    ON e.reportsto = m.employeeid
GROUP BY e.reportsto
HAVING COUNT(m.employeeid) >= 2

 * sqlite:///chinook.db
Done.


[('Andrew', 'Adams'), ('Nancy', 'Edwards'), ('Michael', 'Mitchell')]

__Name the employee who were hired before their boss__

In [86]:
%%sql 
SELECT e.firstname AS employee 
, STRFTIME('%Y.%m', e.hiredate) AS emp_hire_date 
, STRFTIME('%Y-%m', m.hiredate) AS boss_hire_date
FROM employees AS e
LEFT JOIN employees AS m 
    ON e.reportsto = m.employeeid 
GROUP BY e.reportsto 
HAVING e.hiredate < m.hiredate

 * sqlite:///chinook.db
Done.


[('Nancy', '2002.05', '2002-08'), ('Jane', '2002.04', '2002-05')]

__Name the employee whose at least one direct reportee is older than them__

In [73]:
%%sql
SELECT m.firstname AS boss
, e.firstname AS oldest_reportee 
, STRFTIME('%Y', m.birthdate) AS manager_birthdate 
, MAX(STRFTIME('%Y', e.birthdate)) AS oldest_reportee_birthdate
FROM employees AS m  
JOIN employees AS e 
    ON e.reportsto = m.employeeid 
GROUP BY e.reportsto
HAVING m.birthdate > (e.birthdate)

 * sqlite:///chinook.db
Done.


[('Michael', 'Robert', '1973', '1970')]

__Write the manager name and total number of customer supported by their direct report__

In [94]:
%%sql
SELECT m.firstname AS boss 
, COUNT(c.customerid)
FROM employees AS e 
JOIN employees AS m 
    ON e.reportsto = m.employeeid 
JOIN customers AS c 
    ON c.supportrepid = e.employeeid 
GROUP BY e.reportsto 

 * sqlite:///chinook.db
Done.


[('Nancy', 59)]

__Show that only Nancy's direct reports deal with customers__

In [97]:
%%sql
SELECT DISTINCT m.firstname
FROM employees AS e 
JOIN employees AS m 
    ON e.reportsto = m.employeeid 
WHERE e.employeeid IN (
    SELECT DISTINCT supportrepid FROM customers
)

 * sqlite:///chinook.db
Done.


[('Nancy',)]